In [1]:
%load_ext autoreload

In [2]:
%autoreload
%autoreload 2

In [3]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import model_from_json
from mnist_dl_models import load_model
from tensorflow.examples.tutorials.mnist import input_data
from noise_functions_dl import GradientDescentDL

Using TensorFlow backend.


In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
mnist_train_images = mnist.train.images.reshape(-1, 28, 28, 1)
mnist_train_labels = mnist.train.labels
mnist_test_images = mnist.test.images.reshape(-1, 28, 28, 1)
mnist_test_labels = mnist.test.labels
mnist_validation_images = mnist.test.images.reshape(-1, 28, 28, 1)
mnist_validation_labels = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
with tf.Session() as sess:
    models = [load_model(i, 'mnist_dl_models/model_{}_weights.h5'.format(i)) for i in range(3)]
#     print [model.evaluate(mnist_test_images[:10], mnist_test_labels[:10]) for model in models[:3]]
    x = mnist_test_images[:1]
    y = mnist_test_labels[:1]
    dataset_params = [28, 1, 10, (0.0, 1.0)]
    alpha = 3.0
    attack_obj = GradientDescentDL(sess, models, alpha, dataset_params, targeted=False,
                                   batch_size=1, max_iterations=10000, learning_rate=.001,
                                   confidence=0)
    v = attack_obj.attack(x, y, [1.0, 1.0, 1.0])[0]

In [5]:
models = [load_model(i, 'mnist_dl_models/model_{}_weights.h5'.format(i)) for i in range(5)]

In [19]:
def generate_exp_data_dl(num_pts, models, X, Y):
    num_selected = 0
    res_X = []
    res_Y = []
    print len(X)
    for i in xrange(len(X)):
        if sum([model.evaluate(X[i:i + 1], Y[i:i + 1], verbose=0)[1] for model in models]) == len(models):
            res_X.append(X[i])
            res_Y.append(Y[i])
            num_selected += 1
        if num_selected == num_pts:
            break
    if num_selected != num_pts:
        print "not enough points were classified correctly"
    return np.array(res_X), np.array(res_Y)

In [20]:
np.concatenate

<function numpy.core.multiarray.concatenate>

In [21]:
X, Y = generate_exp_data_dl(1000, models, mnist_test_images, mnist_test_labels)

10000
0
500
1000


In [22]:
len(X)

1000

In [23]:
[model.evaluate(np.array(X),np.array(Y)) for model in models]

1000/1000 [==============================] - 0s 391us/step


[[0.0036803178009577097, 1.0],
 [0.0019139794217408054, 1.0],
 [0.0045544123961299195, 1.0],
 [0.024677677407860758, 1.0],
 [0.003316722711431794, 1.0]]

In [25]:
np.save('dl_experiments_data/mnist/X_exp.npy', X)
np.save('dl_experiments_data/mnist/Y_exp.npy', Y)